In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from itertools import combinations
import pandas as pd
import os

def get_csv_files(path, filename):
    full_path = os.path.join(path, filename)
    df=pd.read_csv(full_path)
    ticker=df['ticker'].unique().tolist()
    price_data = df.pivot_table(index='Datetime', 
                            columns='ticker', values='Close')
    return price_data, ticker



In [15]:
# # get_stock_yf(tickers, interval_time, startdate) download stock data for a series of tickers, and adding a intraday return
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR

path = '/Users/jayren/Desktop/stock/Stock_Daily'
dailydata = '/Users/jayren/Desktop/stock/stock_Daily/dailydata'
consolidated = '/stockPrice_consolidated'


#us: us_stock_price.csv
# ca: ca_stock_price.csv
# adhoc: adhoc_stock_price.csv


price_data, tickers = get_csv_files(
    path=os.path.join(dailydata+consolidated), 
    filename='us_stock_price.csv'
)
# return price_data, ticker

tickers=tickers

tickers=['GOOGL', 'VRSN']

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
np.set_printoptions(threshold=np.inf)

data = price_data[tickers]
data.index = pd.to_datetime(data.index)
data = data.asfreq('min')  # 'T' = 1-minute frequency
data = data.dropna()

# Group by date and pivot to get intra-day returns for TD and BMO in separate columns 
#data = filtered_df.pivot_table(index='date', columns='ticker', values='Close')



# Fit a VAR model
model = VAR(data)
results = model.fit(1)  # Fit VAR(1) model

print(results.summary(), '\n\n')

# Step 3: Extract coefficients and recover matrix K
B = results.params.values[1:, :]
I = np.eye(B.shape[0])
K = I - B

print(B, '\n')
print(K, '\n')



# Step 4: Diagonalize K
# Compute eigenvalues and eigenvectors of K
eigenvalues, eigenvectors = np.linalg.eig(K)

# Sort eigenvalues and corresponding eigenvectors by the absolute value of eigenvalues
sorted_indices = np.argsort(np.abs(eigenvalues))[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Step 5: Construct co-integration factor S1
# Transform the original prices using the first eigenvector (strongest mean-reverting component)
prices = data.values  # Original price data matrix
S1 = np.dot(prices, eigenvectors[:, 0])  # Co-integration factor (first eigenvector)

print('eigenvectors for s1 is: ', eigenvectors[:,0])

# Step 6: Analyze the co-integration factor
# Compute autocorrelation of S1 to confirm mean-reverting behavior
def autocorrelation(series, lag):
    return np.corrcoef(series[:-lag], series[lag:])[0, 1]

lags = range(1, 6)  # Test lags from 1 to 5
autocorrelations = [autocorrelation(S1, lag) for lag in lags]

# Display results
print("Eigenvalues:", eigenvalues)
print("Autocorrelations of S1:", autocorrelations)

# Calculate rolling mean and std for S1
rolling_window = 20
mean_S1 = pd.Series(S1).rolling(window=rolling_window).mean()
std_S1 = pd.Series(S1).rolling(window=rolling_window).std()

data.reset_index(inplace=True)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 27, Apr, 2025
Time:                     15:43:17
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -5.48471
Nobs:                     22656.0    HQIC:                  -5.48615
Log likelihood:          -2134.04    FPE:                 0.00414092
AIC:                     -5.48684    Det(Omega_mle):      0.00413982
--------------------------------------------------------------------
Results for equation GOOGL
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            0.332653         0.078151            4.257           0.000
L1.GOOGL         0.999213         0.000190         5246.331           0.000
L1.VRSN         -0.000846         0.000206           -4.115           0.000

Res

/Users/jayren/Desktop/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [16]:
# plot the z-score
#Z-score in pairs trading is a statistical measure that tells you how far the current value of the spread 
# is from its historical mean, measured in standard deviations.
#Z-score Interpretation:
#Action
#Z > +2 Spread is too high, Short the spread (e.g., sell EDU, buy AMC)
# Z < -2 Spread is too low Long the spread (e.g., buy EDU, sell AMC)
# Z near 0 Spread is normal Close positions or stay out
# This strategy assumes that the spread is mean-reverting, so extreme deviations are opportunities to bet on a return to the average
# exit your current position when the z-score is between -0.5 and 0.5



import plotly.graph_objects as go
fig = go.Figure()

# Calculate z-score, This tells you how “extreme” the current spread is relative to its recent behavior.
z_score = (S1 - mean_S1) / std_S1

S1_series=pd.Series(S1, index=data['Datetime'])

z_score_series = (S1-mean_S1) / std_S1
z_score_series.index=data['Datetime']

# Define thresholds for entry and exit
entry_threshold = 2.0
exit_threshold = 0.5

# Generate buy/sell signals
signals = pd.Series(0, index=z_score_series.index)
signals[z_score_series > entry_threshold] = -1  # Sell signal
signals[z_score_series < -entry_threshold] = 1   # Buy signal

upper_exit=exit_threshold
lower_exit=-exit_threshold

buy_signals = z_score_series[signals == 1]
sell_signals = z_score_series[signals == -1]


#Plot z score and entry/exit line
fig.add_trace(go.Scatter(
    x=data['Datetime'], y=z_score,
    mode='lines',
    name='z-score',
    line=dict(color='purple'),
    hovertemplate='%{x|%Y-%m-%d}<br>Z-score: %{y:.2f}<extra></extra>'
))

#entry threshhold lines
fig.add_trace(go.Scatter(x=data['Datetime'], y=[entry_threshold] * len(data['Datetime']),mode='lines',
    name='Entry Threshold',
    line=dict(color='green', dash='dash')
))

fig.add_trace(go.Scatter(x=data['Datetime'], y=[-entry_threshold] * len(data['Datetime']),
    mode='lines',
    line=dict(color='green', dash='dash')
))

# Exit Threshold lines
fig.add_trace(go.Scatter(x=data['Datetime'], y=[exit_threshold] * len(data['Datetime']),mode='lines',
    name='Exit Threshold',
    line=dict(color='orange', dash='dash')
))
fig.add_trace(go.Scatter(x=data['Datetime'], y=[-exit_threshold] * len(data['Datetime']),
    mode='lines',
    line=dict(color='orange', dash='dash')
))

fig.update_layout(title="Z-Score",xaxis_title="Datetime",yaxis_title="Z-Score",
    legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
)

# Buy/Sell signals

fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals,
    mode='markers', name='Buy Signal',
    marker=dict(symbol='triangle-up', color='green', size=15),
    hovertemplate='%{x|%Y-%m-%d}<br>buy signal: %{y:.2f}<extra></extra>'
))
fig.add_trace(go.Scatter(
    x=sell_signals.index, y=sell_signals,
    mode='markers', name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=15),
    hovertemplate='%{x|%Y-%m-%d}<br>Sell Signal: %{y:.2f}<extra></extra>'
))

# existing zone

# fig.add_trace(go.Scatter(
#     x=pd.concat([pd.Series(z_score_series.index), pd.Series(z_score_series.index[::-1])]),
#     y=pd.concat([upper_exit, lower_exit[::-1]]),
#     fill='toself', fillcolor='rgba(173,216,230,0.2)',
#     line=dict(color='rgba(255,255,255,0)'),
#     hoverinfo="skip", showlegend=True, name='existing zone'
# ))

#Show the plot
fig.show(renderer="browser")

In [21]:
#testing new code including s1 line, and so on

# Recalculate rolling mean and std on S1
S1_series = pd.Series(S1, index=data['Datetime'])
rolling_window=195

rolling_mean = S1_series.rolling(window=rolling_window).mean()
rolling_std = S1_series.rolling(window=rolling_window).std()

upper_band = rolling_mean + rolling_std*2
lower_band = rolling_mean - rolling_std*2

# Generate buy/sell signals
signals = pd.Series(0, index=S1_series.index)
signals[S1_series > upper_band] = -1  # Sell signal
signals[S1_series < lower_band] = 1   # Buy signal

buy_signals = S1_series[signals == 1]
sell_signals = S1_series[signals == -1]

# Start building Plotly figure
fig = go.Figure()

# S1 line

fig.add_trace(go.Scatter(
    x=S1_series.index, y=S1_series,
    mode='lines', name='S1 (Spread)', line=dict(color='black'),
    hovertemplate='%{x|%Y-%m-%d}<br>Spread: %{y:.2f}<extra></extra>'
))

# Plot Mean Line
mean_s1 = np.mean(S1) 
fig.add_trace(go.Scatter(x=data['Datetime'], y=[mean_s1] * len(data.index), mode='lines', name='Mean of S1',line=dict(color='red', dash='dash') ))

# Rolling mean
fig.add_trace(go.Scatter(
    x=S1_series.index, y=rolling_mean,
    mode='lines', name='Rolling Mean', line=dict(color='blue', dash='dash')
))

# ±1σ shaded band
fig.add_trace(go.Scatter(
    x=pd.concat([pd.Series(S1_series.index), pd.Series(S1_series.index[::-1])]),
    y=pd.concat([upper_band, lower_band[::-1]]),
    fill='toself', fillcolor='rgba(173,216,230,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip", showlegend=True, name='±1 Std Dev Band'
))

# Buy/Sell signals
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals,
    mode='markers', name='Buy Signal',
    marker=dict(symbol='triangle-up', color='green', size=10)
))
fig.add_trace(go.Scatter(
    x=sell_signals.index, y=sell_signals,
    mode='markers', name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

# Add asset prices

fig.add_trace(go.Scatter(
    x=data['Datetime'], y=data[tickers[0]],
    mode='lines', name=tickers[0], line=dict(color='gray', dash='dot')
))
fig.add_trace(go.Scatter(
    x=data['Datetime'], y=data[tickers[1]],
    mode='lines', name=tickers[1], line=dict(color='purple', dash='dot')
))

# Layout
fig.update_layout(
    title="S1 Co-integration Factor with ±1σ Band, Buy/Sell Signals, and " + tickers[0]+ "/" + tickers[1] + " Prices",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    hovermode="x unified"
)

#fig.write_html("S1_strategy_chart.html")  # Inline if running in a notebook
#Show the plot
fig.show(renderer="browser")